In [2]:
import pexpect
import simplejson as json
from os import makedirs
from os.path import basename, splitext, dirname, join, exists
from glob import glob
from pytezos.michelson.parse import MichelsonParser, michelson_to_micheline

In [9]:
project_dir = dirname(globals()['_dh'][0])
tests_dir = join(project_dir, 'vm', 'tests')
data_dir = join(tests_dir, 'data')
tzt_dir = '/tmp/tzt'

In [11]:
makedirs(data_dir, exist_ok=True)

In [14]:
if not exists(tzt_dir):
    pexpect.run(f'git clone https://gitlab.com/tezos/tzt-reference-test-suite.git {tzt_dir}')

In [15]:
header = """mod runner;

use tezos_vm::Result;
use runner::tzt::TZT;
"""

test_case = """
#[test]
fn {name}() -> Result<()> {{
    TZT::load("{path}")?.run()
}}
"""

In [16]:
ignore = [
    '/tmp/tzt/setdelegate_00.tzt',
    '/tmp/tzt/transfertokens_00.tzt',
    '/tmp/tzt/transfertokens_01.tzt',
    '/tmp/tzt/createcontract_00.tzt',
    '/tmp/tzt/createcontract_01.tzt',
]
# TODO: replace '_' with something that can be parsed with the current grammar
# NOTE: compare_timestamp(00, 01, 02), sub_timestamp_int(04) have incorrect value type (string instead of int)
# NOTE: no need for 'other_contract' section in case of an implicit account

In [17]:
parser = MichelsonParser(extra_primitives=[
    'Contract', 'other_contracts', 'GeneralOverflow', 'Failed', 'MutezUnderflow', 'MutezOverflow'
])
new = {}
sections = [header]
for filename in glob(f'{tzt_dir}/*.tzt'):
    # print(filename)
    if filename in ignore:
        continue
    with open(filename) as f:
        name = splitext(basename(filename))[0].replace('-', '_')
        category = name.split('_')[0]
        full_name = f'tzt_{name}'
        file_json = join(data_dir, 'tzt', f'{full_name}.json')
        if not exists(file_json):
            src = michelson_to_micheline(f.read(), parser=parser)
            tzt = json.dumps(src, indent=2, sort_keys=True)
            with open(file_json, 'w') as fj:
                fj.write(tzt)
              
        sections.append(test_case.format(name=full_name, path=f'{full_name}.json'))
                
#         file_rs = join(tests_dir, f'tzt_{category}.rs')
#         if not exists(file_rs):
#             new[file_rs] = True
#             with open(file_rs, 'w') as fr:
#                 fr.write(header)
#                 fr.write(test_case.format(name=full_name, path=f'{full_name}.json'))
#         elif new.get(file_rs):
#             with open(file_rs, 'a') as fr:
#                 fr.write(test_case.format(name=full_name, path=f'{full_name}.json'))

In [18]:
with open(join(tests_dir, f'tzt.rs'), 'w') as f:
    f.write(''.join(sections))